In [1]:


import ray


ray.init(log_to_driver=False)

2023-06-15 20:15:03,991	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.2.0


# Define Configs

In [2]:
from envs import register_env_dosage

total_workers = 10
num_envs_per_worker = 1

env_name = "dosing-v0"

register_env_dosage(env_name)

[0]
Action: 0
[6.68e+01 9.02e+01 3.64e+01 1.80e-02]
[ True  True  True  True]
[False False False False]
1
0.75
Current Heart Rate: 66.8
Current Blood Pressure: 90.2
Current Temperature: 36.4
Current Dosage: 0.018000000000000002
[0]
Action: 0
[6.39840e+01 9.01560e+01 3.58956e+01 1.96400e-02]
[ True  True False  True]
[False False  True False]
2
0.5
Current Heart Rate: 63.983999999999995
Current Blood Pressure: 90.156
Current Temperature: 35.8956
Current Dosage: 0.01964


/home/hamza/PycharmProjects/example-medicine-delievery-system/venv/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
from ray.rllib.algorithms.ppo import PPOConfig

lstm_model = {"fcnet_hiddens": [32, 32, 32], "vf_share_layers": False, "use_lstm": False,
              "lstm_cell_size": 32, "max_seq_len": 50}
algo = "PPO"
config = (
    PPOConfig()
    .environment(env_name)
    .training(gamma=0.996, num_sgd_iter=3, sgd_minibatch_size=100, clip_param=0.1, lr=1e-3,
              train_batch_size=500,
              entropy_coeff=1e-3, vf_clip_param=10000)
    .resources(num_gpus=1, num_cpus_per_worker=1)
    .rollouts(num_rollout_workers=total_workers, num_envs_per_worker=num_envs_per_worker, enable_connectors=True, observation_filter='MeanStdFilter')
    .framework("torch")
    .training(
        model=lstm_model)
    .evaluation(evaluation_num_workers=1)
)


/home/hamza/PycharmProjects/example-medicine-delievery-system/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [4]:
log_dir = "tmp/pipeline_logs"

# Train RL Agent

The model is trained here using the best config from the tune step. The best training checkpoint is then chosen for evaluation

In [ ]:
from components.train import train_env

train_results = train_env(
    algo=algo,
    config=config,
    log_dir=log_dir,
    iterations=20000,
    stop_reward_mean=1000,
    name="solver",
    checkpoint_frequency=5
)
best_checkpoint = train_results.get_best_result(metric="episode_reward_mean", mode="max").best_checkpoints[0]
best_checkpoint_path = best_checkpoint[0]._local_path

2023-06-15 20:15:08,502	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-15 20:15:08,503	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_dosing-v0_8f7de_00000,500,"{'num_env_steps_sampled': 500, 'num_env_steps_trained': 500, 'num_agent_steps_sampled': 500, 'num_agent_steps_trained': 500}",{},2023-06-15_20-15-25,False,2,{},1.25,1.25,1.25,250,250,6980fb000b5940cf82f596cdca4f4ba0,hamza-Legion-5-15ACH6H,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000007, 'cur_lr': 0.0010000000000000005, 'total_loss': 0.7381738662719727, 'policy_loss': 0.00045662797735227895, 'vf_loss': 0.7390622138977051, 'vf_explained_var': 0.23816063006718954, 'kl': 0.0003322952727709681, 'entropy': 1.4114341338475545, 'entropy_coeff': 0.0010000000000000005}, 'model': {}, 'num_grad_updates_lifetime': 8.0, 'diff_num_grad_updates_vs_sampler_policy': 7.0}}, 'num_env_steps_sampled': 500, 'num_env_steps_trained': 500, 'num_agent_steps_sampled': 500, 'num_agent_steps_trained': 500}",1,192.168.0.97,500,500,500,500,500,500,0,10,0,0,500,"{'cpu_util_percent': 35.3, 'ram_util_percent': 91.6}",278192,{},{},{},"{'mean_raw_obs_processing_ms': 1.602618367064233, 'mean_inference_ms': 1.5779317594042013, 'mean_action_processing_ms': 0.2522529340257831, 'mean_env_wait_ms': 0.5100432564230526, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 1.25, 'episode_reward_min': 1.25, 'episode_reward_mean': 1.25, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 250, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25], 'episode_lengths': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2